This document outlines the steps taken to clean and prepare the dataset for analysis. Below, each decision is explained in the context of these questions and the EDA.

In [ ]:
import pandas as pd

In [ ]:
# Load the dataset to perform data cleaning
file_path = '/mnt/data/cleaned.csv'
data = pd.read_csv(file_path)

I removed the “Unnamed: 0” column, which was an index column automatically created during file export. It does not provide any useful information for our analysis and is not necessary for answering any of our questions. Removing it also streamlined our dataset for analysis, making it easier to search for data via the “name” column.

In [ ]:
# Drop the 'Unnamed: 0' column if it exists
if 'Unnamed: 0' in data.columns:
    data_cleaned = data.drop(columns=['Unnamed: 0'])
else:
    data_cleaned = data.copy()

Rows with all null values were removed. These rows provide no meaningful information for consumer rating or item price, and would add unnecessary size to the dataset without contributing to the analysis.

In [ ]:
# Drop rows with all null values
data_cleaned = data.dropna(how='all')

`Discount_price` and `actual_price` columns were converted from string format (with currency symbols and commas) to numeric values. The `ratings` and `no_of_ratings` columns were also cleaned and converted to numeric, removing commas and handling missing values.

Numeric prices are necessary for performing operations like summing, calculating averages, and identifying the most or least purchased items based on price and discount. By converting them all to a standardised format, it makes it easier for us to conduct analyses. Similarly, numeric ratings allow us to assess the popularity of items within specific categories in a standardised manner.

This helps us to perform data analysis for the following questions:
- Fitness: Determining the best-selling products based on price and discount.
- Travel: Identifying trends in travel-related purchases based on their price.
- Movies/Books: Identifying the most popular films and books based on their ratings and sales.
- Luxury/Beauty: Assessing luxury and discounted purchases using actual and discount prices.


In [ ]:
# Convert price columns to numeric, strip symbols and commas
data_cleaned['discount_price'] = pd.to_numeric(data_cleaned['discount_price'].str.replace('₹', '').str.replace(',', ''), errors='coerce')
data_cleaned['actual_price'] = pd.to_numeric(data_cleaned['actual_price'].str.replace('₹', '').str.replace(',', ''), errors='coerce')

# Convert ratings and number of ratings to numeric
data_cleaned['ratings'] = pd.to_numeric(data_cleaned['ratings'], errors='coerce')
data_cleaned['no_of_ratings'] = pd.to_numeric(data_cleaned['no_of_ratings'].str.replace(',', ''), errors='coerce')

Outliers in the `discount_price` column were removed using the IQR (Interquartile Range) method. The lower and upper bounds were calculated based on 1.5 times the IQR https://www.khanacademy.org/math/statistics-probability/summarizing-quantitative-data/box-whisker-plots/a/identifying-outliers-iqr-rule.

Outliers in pricing can significantly skew analysis, particularly when analysing sales trends. Removing extreme values will help us focus on the typical price range for products, leading to more accurate insights into best-selling products and popular discounts. Furthermore, when assessing which luxury items or discounted items were the most popular, removing outliers ensures that extreme price points (which may be errors or anomalies) don't distort our results.

In [ ]:
# Outlier removal using IQR method for prices
Q1 = data_cleaned['discount_price'].quantile(0.25)
Q3 = data_cleaned['discount_price'].quantile(0.75)
IQR = Q3 - Q1

# Define lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Removing outliers from the discount_price column
data_cleaned = data_cleaned[(data_cleaned['discount_price'] >= lower_bound) & (data_cleaned['discount_price'] <= upper_bound)]

In [ ]:
# Save the cleaned dataset
cleaned_file_path = '/mnt/data/final_cleaned_dataset.csv'
data_cleaned.to_csv(cleaned_file_path, index=False)